<a href="https://colab.research.google.com/github/akbarroyhan69-gif/running-pace-analyst-/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [2]:
import pandas as pd
import numpy as np


# Unggah Dataset ke Google Colab



In [3]:
from google.colab import files
uploaded = files.upload()


Saving s.xlsx to s.xlsx


Pada tahap ini digunakan modul files dari google.colab untuk melakukan proses unggah (upload) dataset ke lingkungan Google Colab. Perintah files.upload() memungkinkan peneliti mengunggah file dataset secara langsung dari perangkat lokal ke sesi Colab yang sedang digunakan.
Dataset yang diunggah merupakan data hasil kuesioner mahasiswa Universitas Darussalam (UNIDA) yang berisi variabel-variabel yang berkaitan dengan karakteristik mahasiswa dan kebiasaan aktivitas lari, yang selanjutnya digunakan dalam analisis dan pemodelan pace lari. Setelah proses unggah dilakukan, file tersebut akan tersimpan sementara di direktori kerja Colab dan dapat diakses menggunakan library pandas untuk tahap pengolahan data berikutnya.
Tahap ini sangat penting karena menjadi langkah awal dalam integrasi data penelitian ke dalam sistem, sehingga seluruh proses pembersihan data, analisis, dan pembangunan model Random Forest dapat dilakukan secara terstruktur dan sistematis.

# Membaca dan Membersihkan Dataset

In [4]:
d=pd.read_excel('s.xlsx')
d=d.dropna(how='all')

Pada tahap ini dilakukan proses pembacaan dataset penelitian menggunakan fungsi read_excel() dari library pandas. File s.xlsx merupakan dataset yang berisi data hasil kuesioner mahasiswa Universitas Darussalam (UNIDA) yang mencakup berbagai variabel yang berkaitan dengan karakteristik individu dan kebiasaan aktivitas lari mahasiswa, yang digunakan sebagai dasar analisis pace lari.

Selanjutnya, dilakukan proses pembersihan data dengan menggunakan perintah dropna(how='all'). Perintah ini bertujuan untuk menghapus baris data yang seluruh nilai pada kolomnya kosong (missing values). Langkah ini dilakukan untuk memastikan bahwa hanya data yang memiliki informasi relevan yang digunakan dalam proses analisis dan pemodelan, sehingga dapat meningkatkan kualitas dataset dan mengurangi potensi gangguan dalam proses pelatihan model.

# Standarisasi Format Data

In [7]:
d["pace"] = d["pace"].astype(str).str.replace(",", ".")
d["semester"] = d["semester"].astype(str).str.replace("semester","")



Pada tahap ini dilakukan proses standarisasi dan penyesuaian format data pada beberapa variabel penting agar dapat diproses secara numerik oleh sistem machine learning.

Baris kode d["pace"] = d["pace"].astype(str).str.replace(",", ".") digunakan untuk mengubah nilai pada variabel pace lari ke dalam format teks sementara, kemudian mengganti tanda koma (,) menjadi tanda titik (.). Langkah ini dilakukan karena perbedaan format penulisan desimal pada data kuesioner, sehingga perlu diseragamkan agar nilai pace dapat dikonversi ke bentuk numerik secara benar pada tahap selanjutnya.

Selanjutnya, baris kode d["semester"] = d["semester"].astype(str).str.replace("semester","") digunakan untuk membersihkan variabel semester dengan menghapus teks non-numerik “semester”. Proses ini bertujuan agar nilai semester hanya berisi angka, sehingga dapat digunakan sebagai variabel numerik dalam analisis dan pemodelan.

# Konversi Tipe Data dan Penanganan Nilai Kosong


In [8]:
# Konversi tipe numeric
num_cols = ["semester", "usia", "tinggi", "latihan_per_minggu", "berat", "pace", "perokok"]

for col in num_cols:
    d[col] = pd.to_numeric(d[col], errors="coerce")

# Kolom int (kecuali pace dan perokok)
cols_to_int = [col for col in num_cols if col not in ['pace', 'perokok']]

# Tangani NaN
d[cols_to_int] = d[cols_to_int].fillna(0).astype('int64')

# Ubah perokok menjadi boolean
d['perokok'] = d['perokok'].fillna(0).astype(bool)


Pada tahap ini dilakukan proses konversi tipe data numerik serta penanganan nilai kosong (missing values) pada dataset hasil kuesioner mahasiswa Universitas Darussalam (UNIDA). Proses ini bertujuan untuk memastikan seluruh variabel dapat diproses dengan baik oleh algoritma machine learning.

*   Variabel yang bersifat numerik, yaitu semester, usia, tinggi badan, frekuensi latihan per minggu, berat badan, pace lari, dan status perokok, dikonversi ke tipe numerik menggunakan fungsi to_numeric() dengan parameter errors="coerce". Parameter ini berfungsi untuk mengubah nilai yang tidak dapat dikonversi menjadi numerik menjadi nilai kosong (NaN), sehingga memudahkan proses pembersihan data selanjutnya.
*   Selanjutnya, dilakukan pemisahan kolom yang akan dikonversi ke tipe bilangan bulat (integer), yaitu seluruh variabel numerik kecuali pace lari dan status perokok. Nilai kosong pada kolom-kolom tersebut diisi dengan nilai nol, kemudian dikonversi ke tipe data int64. Langkah ini dilakukan karena variabel seperti semester, usia, tinggi badan, berat badan, dan frekuensi latihan secara konseptual merupakan data diskrit.


*   Untuk variabel status perokok, nilai kosong diisi dengan nol dan kemudian dikonversi ke tipe data boolean (True/False). Konversi ini bertujuan untuk merepresentasikan status perokok sebagai variabel biner, sehingga lebih sesuai untuk digunakan dalam proses pemodelan dan analisis faktor yang memengaruhi pace lari mahasiswa.

Tahap konversi tipe data dan penanganan nilai kosong ini merupakan bagian penting dari data preprocessing, karena memastikan konsistensi dan validitas data, serta meningkatkan keandalan model Random Forest dalam menganalisis dan memprediksi pace lari mahasiswa UNIDA.



# Perhitungan Indeks Massa Tubuh (IMT)

In [9]:
d['tinggi_m'] = d['tinggi'] / 100
d['imt'] = d['berat'] / (d['tinggi_m'] ** 2)

Pada tahap ini dilakukan proses pembentukan variabel turunan (feature engineering) berupa Indeks Massa Tubuh (IMT) yang digunakan untuk menggambarkan komposisi tubuh mahasiswa.

Baris kode d['tinggi_m'] = d['tinggi'] / 100 digunakan untuk mengonversi tinggi badan mahasiswa dari satuan sentimeter ke meter. Konversi ini diperlukan agar perhitungan IMT dapat dilakukan sesuai dengan rumus standar yang menggunakan satuan meter.

Selanjutnya, baris kode d['imt'] = d['berat'] / (d['tinggi_m'] ** 2) digunakan untuk menghitung nilai Indeks Massa Tubuh dengan membagi berat badan (kilogram) dengan kuadrat tinggi badan (meter). Nilai IMT yang dihasilkan mencerminkan proporsi berat badan terhadap tinggi badan dan digunakan sebagai salah satu variabel independen dalam analisis faktor-faktor yang memengaruhi pace lari mahasiswa Universitas Darussalam (UNIDA).

Pembentukan variabel IMT ini bertujuan untuk memperkaya informasi dalam dataset dan membantu model Random Forest dalam menangkap hubungan antara kondisi fisik mahasiswa dengan performa lari, khususnya dalam hal pace lari.




# Augmentasi Data

In [10]:
# ---------------------------
# AUGMENTASI
# ---------------------------
def augment_row(row, n=5):
    augmented = []
    for _ in range(n):
        new_row = row.copy()
        new_row["pace"] += np.random.uniform(0, 0.3)
        new_row["pace"] = round(new_row["pace"], 2)
        new_row["berat"] += np.random.randint(0, 5)
        augmented.append(new_row)
    return augmented

augmented_rows = []
for i in range(len(d)):
    augmented_rows.extend(augment_row(d.iloc[i], n=5))

augmented_df = pd.DataFrame(augmented_rows)

Pada tahap ini dilakukan augmentasi data untuk meningkatkan jumlah dan variasi data latih yang digunakan dalam pemodelan pace lari mahasiswa Universitas Darussalam (UNIDA). Augmentasi data dilakukan karena jumlah data asli relatif terbatas, sehingga diperlukan penambahan variasi data sintetis agar model Random Forest dapat belajar dengan lebih baik dan mengurangi risiko overfitting.

Fungsi augment_row() digunakan untuk menghasilkan beberapa data baru dari satu baris data asli. Pada setiap proses augmentasi, nilai pace lari dimodifikasi dengan menambahkan nilai acak kecil dalam rentang 0 hingga 0,3 menit per kilometer. Penambahan ini bertujuan untuk mensimulasikan variasi alami performa lari mahasiswa yang dapat dipengaruhi oleh kondisi fisik dan lingkungan. Nilai pace kemudian dibulatkan hingga dua angka desimal agar tetap realistis.

Selain itu, nilai berat badan juga dimodifikasi dengan menambahkan bilangan bulat acak antara 0 hingga 4 kilogram. Perubahan ini dimaksudkan untuk merepresentasikan variasi ringan pada berat badan mahasiswa yang masih berada dalam rentang wajar dan dapat terjadi dalam kondisi nyata, misalnya akibat perbedaan waktu pengukuran atau kondisi tubuh.

Proses augmentasi dilakukan sebanyak lima kali untuk setiap data asli, sehingga setiap entri data menghasilkan lima data baru. Seluruh data hasil augmentasi kemudian digabungkan ke dalam sebuah DataFrame baru (augmented_df) yang digunakan sebagai dataset tambahan dalam proses pemodelan.

Selanjutnya, dilakukan pembentukan variabel target kategori pace, yaitu cepat dan lambat. Kategori cepat diberikan untuk nilai pace kurang dari 5 menit per kilometer, sedangkan kategori lambat diberikan untuk nilai pace sama dengan atau lebih dari 5 menit per kilometer. Pengelompokan ini digunakan untuk mendukung analisis klasifikasi pace lari mahasiswa.

Tahap augmentasi data ini berperan penting dalam meningkatkan keberagaman data, memperkuat kemampuan generalisasi model Random Forest, serta menghasilkan model yang lebih stabil dan andal dalam menganalisis faktor-faktor yang memengaruhi pace lari mahasiswa UNIDA.

# Penggabungan Data Asli dan Data Hasil Augmentasi

In [11]:
full_df = pd.concat([d, augmented_df], ignore_index=True)
full_df = full_df.dropna()

Pada tahap ini dilakukan proses penggabungan dataset asli dengan dataset hasil augmentasi untuk membentuk dataset akhir yang digunakan dalam proses pemodelan. Penggabungan dilakukan menggunakan fungsi pd.concat() dengan parameter ignore_index=True, sehingga seluruh data, baik data asli maupun data hasil augmentasi, disatukan dalam satu struktur data tanpa mempertahankan indeks lama.

Dataset gabungan (full_df) kemudian melalui proses pembersihan lanjutan dengan menggunakan fungsi dropna(). Langkah ini bertujuan untuk menghapus baris data yang masih mengandung nilai kosong (missing values) setelah proses penggabungan, sehingga dataset akhir hanya berisi data yang lengkap dan valid.

Tahap ini sangat penting karena memastikan bahwa model Random Forest dilatih menggunakan dataset yang lebih besar, lebih bervariasi, dan bebas dari nilai kosong. Dengan demikian, kualitas data latih meningkat, risiko kesalahan dalam proses pelatihan model dapat diminimalkan, serta hasil analisis faktor-faktor yang memengaruhi pace lari mahasiswa Universitas Darussalam (UNIDA) menjadi lebih akurat dan dapat diandalkan.

In [12]:
full_df.to_csv("full_df.csv", index=False)
d.to_csv("data_d.csv", index=False)

In [13]:
from google.colab import files

files.download("full_df.csv")
files.download("data_d.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>